# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
# import libraries
import pandas as pd
import re

import pickle

from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report

from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC

import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])

%load_ext line_profiler

In [ ]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')
with engine.connect() as connection:
    df = pd.read_sql_table(table_name='messages', con=connection)

df.replace(2, 1, inplace=True)

X = df.loc[:, 'message']
y = df.iloc[:, 4:]

In [ ]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

### 2. Write a tokenization function to process your text data

In [ ]:
y.sum()

In [ ]:
def tokenize(text):
    text = re.sub(r"[^A-Za-z0-9\s]", " ", text)
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    words = word_tokenize(text.lower().strip())
    words = [WordNetLemmatizer().lemmatize(w) for w in words if w not in stopwords.words("english")]
    
    return words

In [ ]:
tokenize(X[355])

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(LinearSVC()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.8, random_state=42)

In [ ]:
%%time

#%lprun -f tokenize 
pipeline.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
%%time
pipeline.score(X_test, y_test)

In [ ]:
y_pred = pipeline.predict(X_test)

In [ ]:
y_pred_df = pd.DataFrame(y_pred)
y_pred_df.columns = y_test.columns.astype('str')

In [ ]:
# colname = 'related'
# print(classification_report(y_test[colname], y_pred_df[colname]))

In [ ]:
#for column in y_test.columns.astype('str'):
#    print(classification_report(y_test[column], y_pred_df[column]))

In [ ]:
print(classification_report(y_test, y_pred_df, target_names=y_test.columns))

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
pipeline.get_params()

In [ ]:
parameters = {
    'vect__tokenizer': [None, tokenize],
    'clf__estimator__multi_class': ['ovr', 'crammer_singer'],
    'clf__estimator__max_iter': [2000, 8000, 15000],
    'clf__estimator__dual':[False, True]
}
cv = GridSearchCV(pipeline, param_grid=parameters, verbose=True)

In [ ]:
cv.fit(X_train, y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
cv.best_params_

In [ ]:
cv.score(X_test, y_test)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
pipeline8 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [ ]:
pipeline8.fit(X_train, y_train)

In [ ]:
pipeline8_pred_test_df = pd.DataFrame(pipeline8.predict(X_test))

In [ ]:
print(classification_report(y_test, pipeline8_pred_test_df, target_names=y_test.columns))

### 9. Export your model as a pickle file

In [ ]:
with open('message-classifier.pkl', 'wb') as message_classifier_file:
    pickle.dump(cv.estimator, message_classifier_file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.